**Jupyter Notebook: Analysis of data from the Review, Tenure and Promotion project**

*Dr. Juan Pablo Alperin, Scholarly Communications Lab*

------------------

This is the code used to analyze the data found in the following paper: 

Alperin, J., Muñoz Nieves, C., Schimanski, L., Fischman, G., Niles, M., and McKiernan, E. (2018). How significant are the public dimensions of faculty work in review, promotion, and tenure documents? *Humanities Commons [preprint]* [doi.org/10.17613/M6W950N3](https://doi.org/10.17613/M6W950N35)

The input file itself can be found here: 

Alperin, Juan Pablo; Muñoz Nieves, Carol; Schimanski, Lesley; McKiernan, Erin C.; Niles, Meredith T., 2018, "Terms and Concepts found in Tenure and Promotion Guidelines from the US and Canada", [doi.org/10.7910/DVN/VY4TJE](https://doi.org/10.7910/DVN/VY4TJE), *Harvard Dataverse*, V1, UNF:6:RD6O/u5UKt8wHog/sJUx9Q== [fileUNF] 

In [20]:
import numpy as np
import pandas as pd

from IPython.display import display, HTML

percent_float = '{:.0f}%'.format
normal_float = '{:.0f}'.format
long_float = '{:.4f}'.format

# Save the outputs to an excel
writer = pd.ExcelWriter('RPT_summaries.xlsx')

In [21]:
# Read the dataset. Available from: https://doi.org/10.7910/DVN/VY4TJE
df = pd.read_csv('RPT_final_dataset.csv')

In [22]:
# Some columns are descriptors, the rest are categories of interest
descriptors = ['level', 'country', 'inst_type', 'inst_subtype', 'uni_name', 'label', 'discipline']
categories = [x for x in df.columns if x not in descriptors]
    
# make a list of the first four to do some combined analysis
output_types = [x for x in categories if 'output_type_' in x and 'generic' not in x]
output_types_traditional = [x for x in output_types if 'performances' not in x and 'exhibitions' not in x]
output_types_new = [x for x in categories if 'outputs_' in x]

publication_venues = [x for x in categories if 'publication_venue_' in x ]

metrics = [x for x in categories if 'metrics_' in x]

peer_reviews = [x for x in categories if 'peer_review_' in x]

impact_factor = [x for x in categories if 'if_' in x]

In [23]:
# Aggregate some metrics
df['any_metric'] = df[metrics].apply(lambda x: x.any(), axis=1)
df['any_output_type'] = df[output_types].apply(lambda x: x.any(), axis=1)
df['any_output_type_traditional'] = df[output_types].apply(lambda x: x.any(), axis=1)
df['any_publication_venue'] = df[publication_venues].apply(lambda x: x.any(), axis=1)
df['any_peer_review'] = df[peer_reviews].apply(lambda x: x.any(), axis=1)

# new output analysis
output_categories = set([c.split('_')[1] for c in output_types_new])
for cat in output_categories:
    outputs_in_category = [x for x in output_types_new if ('outputs_%s') % cat in x]
    df['any_outputs_%s' % cat] = df[outputs_in_category].apply(lambda x: x.any(), axis=1)
    categories += ['any_outputs_%s' % cat]

# remove the metrics_ from the list now that they are integrated
# categories = list(set(categories).difference(metrics)) 
categories += ['any_metric', 'any_output_type', 'any_output_type_traditional', 'any_publication_venue', 'any_peer_review']

# Considering mentions of IF and of High Impact Journals together
df['any_impact_factor'] = df[['metrics_impact_factor', 'metrics_high_impact_journals']].apply(lambda x: x.any(), axis=1)
categories += ['any_impact_factor']

In [24]:
# Drop any nulls, means we did not have document
# Currently only drops Harvard University institutional-level doc
df = df[~df[categories[0]].isnull()]

df = df[descriptors].join(df[categories].astype(bool))

In [25]:
# Make a new DF for mentions in UN, AU, UN or AU, AU but not UN (aggregations)
uni_or_au = df.groupby('uni_name').agg(lambda x: x.any())  # UN or at least one AU
print(len(uni_or_au))

uni = df[df.level == 'UN'] # UN only
print(len(uni))

au = df[df.level == 'AU'] # AUs all (AU-level analysis)
print(len(au))

# at least one AU
au_uni = au.groupby('uni_name').agg(lambda x: x.any())  # returns 60 institutions
au_uni = uni[descriptors].set_index('uni_name').join(au_uni[categories]) # has all 129
print(len(au_uni))

# at least one AU and NOT at UN level
au_not_uni = uni.set_index('uni_name').join(au_uni[categories], lsuffix='_un', rsuffix='_au')
for cat in categories:
    au_not_uni[cat] = au_not_uni[cat+'_un'] & au_not_uni[cat+'_au']
    del au_not_uni[cat+'_un']
    del au_not_uni[cat+'_au']
print(len(au_not_uni))

129
129
382
129
129


In [26]:
def summarize_categories(df, name = 'n', multiply=False):
    tmp = df[categories].sum() #.divide(len(df))
    tmp.name = name
    tmp_df = pd.DataFrame(tmp)

    if multiply:
        m=100
    else:
        m=1
    tmp_df[tmp.name + '_p'] = tmp.divide(len(df)/m)
    return tmp_df


In [27]:
pd.options.display.float_format = normal_float

# Produce a sheet that has all the above
final = summarize_categories(uni_or_au, 'UN+AUs uni level').join(
        summarize_categories(uni, 'UN uni level')).join(
        summarize_categories(au_uni, 'AUs uni level')).join(
        summarize_categories(au_not_uni, 'AUs not UN, uni level')).join(
        summarize_categories(au, 'AUs au level'))
# final.multiply(100)
final.to_clipboard()

final.to_excel(writer, 'all')

In [28]:
# IF analysis
def add_if_calculations(df_if):
    df_if.loc[:,'if_specified_or_unspecified'] = df_if.apply(lambda row: row['if_specified'] or row['if_unspecified'], axis=1)
    df_if.loc[:,'if_specified_and_unspecified'] = df_if.apply(lambda row: row['if_specified'] and row['if_unspecified'], axis=1)
    return df_if

uni_or_au = add_if_calculations(uni_or_au)
uni = add_if_calculations(uni)
au = add_if_calculations(au)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
pd.options.display.float_format = normal_float
x = uni_or_au.groupby('inst_type')
# print(x.size())
# print(x.sum().transpose())
# x.sum().transpose().to_clipboard()
# # x.size().to_clipboard()
# print(x)

print(x.size())
y = x.sum().transpose()
y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))
cols = sorted(y.columns, reverse=True)

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[print_cols].to_clipboard()
print(y[sorted(cols, reverse=True)])
y[print_cols].to_excel(writer, "inst_type")

inst_type
Bacc-type    33
M-type       38
R-type       58
dtype: int64
inst_type                                           R-type  M-type  Bacc-type
N                                                       58      38         33
any_impact_factor                                       24       8          0
any_metric                                              44      15          5
any_output_type                                         54      36         31
any_output_type_traditional                             54      36         31
any_outputs_arts                                        48      32         29
any_outputs_data                                        10       4          2
any_outputs_education                                   53      32         30
any_outputs_events                                      53      34         29
any_outputs_generic                                     56      35         30
any_outputs_ict                                         45      19     

In [30]:
pd.options.display.float_format = normal_float
x = uni_or_au[uni_or_au.inst_subtype.isin(['R1', 'R2', 'R3', 'U15'])].groupby('inst_subtype')
print(x.size())
y = x.sum().transpose()

y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[sorted(y.columns)].to_clipboard()
print(y[cols])
y[print_cols].to_excel(writer, "inst_subtype (R-types)")


inst_subtype
R1    16
R2    15
R3    14
dtype: int64
inst_subtype                                        R3  R2  R1
N                                                   14  15  16
any_impact_factor                                    5   7   7
any_metric                                           8  11  15
any_output_type                                     13  14  15
any_output_type_traditional                         13  14  15
any_outputs_arts                                     9  13  14
any_outputs_data                                     1   2   5
any_outputs_education                               11  15  16
any_outputs_events                                  11  14  16
any_outputs_generic                                 12  15  16
any_outputs_ict                                      9  13  14
any_outputs_ip                                       8  11  13
any_outputs_preprints                                1   4   2
any_outputs_public                                   8  12  13
an

In [31]:
# Disciplines at the academic unit level
# Get only R-type

print("R-type only")
print("-----------")
x = au[au.inst_type == 'R-type'].groupby(['uni_name', 'discipline']).agg(lambda x: x.any())  # returns 60 institutions
x.index = x.index.droplevel(0)
x = x.reset_index().groupby('discipline')

print(x.size())
y = x.sum().transpose()
y.loc['N'] = x.size().tolist()
y = y.reindex(['N'] + sorted([c for c in y.index if c != 'N']))

cols = sorted(y.columns, reverse=True)
print_cols = []
for c in cols: 
    p_col = "%s_p" % c
    y[p_col] = y[c]/x.size()[c]
    print_cols += [c, p_col]
# print(x.sum().transpose()['R1'].divide(2))
# x.sum().transpose().to_clipboard()
y[print_cols].to_clipboard()
print(y[sorted(cols, reverse=True)])

y[print_cols].to_excel(writer, "disciplines (R-types)")

R-type only
-----------
discipline
Interdisciplinary                  23
Life Sciences                      33
Physical Sciences & Mathematics    21
Social Sciences and Humanities     39
dtype: int64
discipline                                          Social Sciences and Humanities  \
N                                                                               39   
any_impact_factor                                                                8   
any_metric                                                                      25   
any_output_type                                                                 37   
any_output_type_traditional                                                     37   
any_outputs_arts                                                                30   
any_outputs_data                                                                 3   
any_outputs_education                                                           35   
any_outputs_events        

In [32]:
# save these summaries
writer.save()

In [ ]:
# Use this to cross any two categoreis

pd.options.display.float_format = percent_float
cat1 = 'any_metric'
cat2 = 'public_and/or_community_engagement'

df = uni_or_au[uni_or_au.inst_type == 'R-type']
df2 = df[[cat1, cat2]]
df2 = pd.crosstab(df2[cat1], df2[cat2]) # .apply(lambda x: round(x*100/len(df2)))
df2.to_clipboard()
print(df2)
print('------------')
print("N=%s" % len(df))
print("%s == TRUE: %s; FALSE == %s" % (cat1, df[cat1].sum(), len(df)-df[cat1].sum()))
print("%s == TRUE: %s; FALSE == %s" % (cat2, df[cat2].sum(), len(df)-df[cat2].sum()))